In [1]:
import pandas as pd
import tqdm.auto as tqdm

complete_to_two_digits = lambda x: str(x).zfill(2) if x < 10 else str(x)

def read_csv_file(user_id, video_id):
    filepath = f'../Formated_Data/Experiment_1/{user_id}/video_{video_id}.csv'
    #print(filepath)
    df = pd.read_csv(filepath)
    
    return df

csv_data = []
for user_id in range(1, 49):
    user_data = []
    for video_id in range(0, 9):

        df = read_csv_file(user_id, video_id)
 
        df['user_id'] = user_id # 添加标签（用户ID）
        df['video_id'] = video_id # 添加video_id字段
        #df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
        df['timestamp_floor'] = pd.to_datetime(df['Timestamp']).dt.floor('100ms')
        user_data.append(df)

    user_data_combined = pd.concat(user_data, ignore_index=True)
    csv_data.append(user_data_combined)

csv_data = pd.concat(csv_data, ignore_index=True)


In [2]:
csv_data.dtypes

Timestamp                   object
PlaybackTime               float64
UnitQuaternion.x           float64
UnitQuaternion.y           float64
UnitQuaternion.z           float64
UnitQuaternion.w           float64
HmdPosition.x              float64
HmdPosition.y              float64
HmdPosition.z              float64
user_id                      int64
video_id                     int64
timestamp_floor     datetime64[ns]
dtype: object

In [3]:
from datetime import timedelta
import numpy as np
import random
from tqdm.auto import tqdm

def process_data(csv_data):

    data_0, data_1, data_2, data_3, data_4, data_5, data_6, data_7, data_8 = [], [], [], [], [], [], [], [], []
    labels_0, labels_1, labels_2, labels_3, labels_4, labels_5, labels_6, labels_7, labels_8 = [], [], [], [], [], [], [], [], []

    for user_id, user_data in tqdm(csv_data.groupby('user_id')):
        for video_id, video_data in user_data.groupby('video_id'):
            for _, floor_data in video_data.groupby(video_data['timestamp_floor']):
                floor_data = floor_data[['UnitQuaternion.x', 'UnitQuaternion.y', 'UnitQuaternion.z', 'UnitQuaternion.w', 'HmdPosition.x', 'HmdPosition.y', 'HmdPosition.z']]
                if video_id == 0:
                    data_0.append(round(floor_data.mean(), 5).values)
                    labels_0.append(user_id)
                elif video_id == 1:
                    data_1.append(round(floor_data.mean(), 5).values)
                    labels_1.append(user_id)
                elif video_id == 2:
                    data_2.append(round(floor_data.mean(), 5).values)
                    labels_2.append(user_id)
                elif video_id == 3:
                    data_3.append(round(floor_data.mean(), 5).values)
                    labels_3.append(user_id)
                elif video_id == 4:
                    data_4.append(round(floor_data.mean(), 5).values)
                    labels_4.append(user_id)
                elif video_id == 5:
                    data_5.append(round(floor_data.mean(), 5).values)
                    labels_5.append(user_id)
                elif video_id == 6:
                    data_6.append(round(floor_data.mean(), 5).values)
                    labels_6.append(user_id)
                elif video_id == 7:
                    data_7.append(round(floor_data.mean(), 5).values)
                    labels_7.append(user_id)
                elif video_id == 8:
                    data_8.append(round(floor_data.mean(), 5).values)
                    labels_8.append(user_id)

    return np.array(data_0), np.array(labels_0), np.array(data_1), np.array(labels_1), \
            np.array(data_2), np.array(labels_2), np.array(data_3), np.array(labels_3), \
            np.array(data_4), np.array(labels_4), np.array(data_5), np.array(labels_5), \
            np.array(data_6), np.array(labels_6), np.array(data_7), np.array(labels_7), \
            np.array(data_8), np.array(labels_8)

In [4]:
data_0, labels_0, data_1, labels_1, data_2, labels_2, data_3, labels_3, data_4, labels_4, data_5, labels_5, data_6, labels_6, data_7, labels_7, data_8, labels_8 = process_data(csv_data)

  0%|          | 0/48 [00:00<?, ?it/s]

In [5]:
print('data_0.shape: {}'.format(data_0.shape))
print('labels_0.shape: {}'.format(labels_0.shape))
print('data_1.shape: {}'.format(data_1.shape))
print('labels_1.shape: {}'.format(labels_1.shape))
print('data_2.shape: {}'.format(data_2.shape))
print('labels_2.shape: {}'.format(labels_2.shape))
print('data_3.shape: {}'.format(data_3.shape))
print('labels_3.shape: {}'.format(labels_3.shape))
print('data_4.shape: {}'.format(data_4.shape))
print('labels_4.shape: {}'.format(labels_4.shape))
print('data_5.shape: {}'.format(data_5.shape))
print('labels_5.shape: {}'.format(labels_5.shape))
print('data_6.shape: {}'.format(data_6.shape))
print('labels_6.shape: {}'.format(labels_6.shape))
print('data_7.shape: {}'.format(data_7.shape))
print('labels_7.shape: {}'.format(labels_7.shape))
print('data_8.shape: {}'.format(data_8.shape))
print('labels_8.shape: {}'.format(labels_8.shape))


data_0.shape: (78978, 7)
labels_0.shape: (78978,)
data_1.shape: (97550, 7)
labels_1.shape: (97550,)
data_2.shape: (140903, 7)
labels_2.shape: (140903,)
data_3.shape: (83017, 7)
labels_3.shape: (83017,)
data_4.shape: (98922, 7)
labels_4.shape: (98922,)
data_5.shape: (314629, 7)
labels_5.shape: (314629,)
data_6.shape: (217545, 7)
labels_6.shape: (217545,)
data_7.shape: (80601, 7)
labels_7.shape: (80601,)
data_8.shape: (143948, 7)
labels_8.shape: (143948,)


In [17]:
import numpy as np
np.set_printoptions(linewidth=np.inf)
print(data_0[78900:78910])
print(labels_0[78900:78910])

[[ 0.042   -0.57    -0.03     0.82    -0.153    1.15711 -0.20656]
 [ 0.04233 -0.57    -0.03     0.82    -0.153    1.157   -0.206  ]
 [ 0.043   -0.57    -0.03     0.82    -0.15322  1.157   -0.206  ]
 [ 0.04311 -0.57    -0.03     0.82    -0.15356  1.15711 -0.206  ]
 [ 0.04356 -0.57    -0.02989  0.82    -0.15333  1.15789 -0.206  ]
 [ 0.04333 -0.57    -0.03     0.82    -0.15333  1.15767 -0.206  ]
 [ 0.043   -0.57    -0.02989  0.82    -0.153    1.157   -0.206  ]
 [ 0.043   -0.57078 -0.02922  0.81944 -0.15344  1.157   -0.206  ]
 [ 0.043   -0.571   -0.02944  0.819   -0.153    1.157   -0.206  ]
 [ 0.043   -0.571   -0.03     0.819   -0.153    1.157   -0.20633]]
[48 48 48 48 48 48 48 48 48 48]


In [18]:
def append_to_file(input_file, output_file, additional_content):
    with open(input_file, 'r') as f:
        original_content = f.read()

    new_content = original_content + '\n' + additional_content

    with open(output_file, 'w') as f:
        f.write(new_content)


In [19]:
def initTimeNetData(data, label, step):
    append_string = ""
    for i in range(0, len(label), step):
        group = label[i:i+step]
        if len(set(group)) == 1:
            temp_transposed_matrix = np.transpose(data[i:i+step])
            append_string += ":".join([",".join(map(str, row)) for row in temp_transposed_matrix])
            append_string += ":" + str(group[0]) + "\n"

    return append_string

In [22]:
for i in range(9):
    train_data = []
    train_labels = []
    test_data = []
    test_labels = []
    if i == 0:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_1, data_2, data_3, data_4, data_5, data_6, data_7, data_8))
        train_labels = np.hstack((labels_1, labels_2, labels_3, labels_4, labels_5, labels_6, labels_7, labels_8))
        test_data = data_0
        test_labels = labels_0
        print(f'i: {i}, initiate data finished.')
    elif i == 1:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_2, data_3, data_4, data_5, data_6, data_7, data_8))
        train_labels = np.hstack((labels_0, labels_2, labels_3, labels_4, labels_5, labels_6, labels_7, labels_8))
        test_data = data_1
        test_labels = labels_1
        print(f'i: {i}, initiate data finished.')
    elif i == 2:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_3, data_4, data_5, data_6, data_7, data_8))
        train_labels = np.hstack((labels_0, labels_1, labels_3, labels_4, labels_5, labels_6, labels_7, labels_8))
        test_data = data_2
        test_labels = labels_2
        print(f'i: {i}, initiate data finished.')
    elif i == 3:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_2, data_4, data_5, data_6, data_7, data_8))
        train_labels = np.hstack((labels_0, labels_1, labels_2, labels_4, labels_5, labels_6, labels_7, labels_8))
        test_data = data_3
        test_labels = labels_3
        print(f'i: {i}, initiate data finished.')
    elif i == 4:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_2, data_3, data_5, data_6, data_7, data_8))
        train_labels = np.hstack((labels_0, labels_1, labels_2, labels_3, labels_5, labels_6, labels_7, labels_8))
        test_data = data_4
        test_labels = labels_4
    elif i == 5:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_2, data_3, data_4, data_6, data_7, data_8))
        train_labels = np.hstack((labels_0, labels_1, labels_2, labels_3, labels_4, labels_6, labels_7, labels_8))
        test_data = data_5
        test_labels = labels_5
        print(f'i: {i}, initiate data finished.')
    elif i == 6:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_2, data_3, data_4, data_5, data_7, data_8))
        train_labels = np.hstack((labels_0, labels_1, labels_2, labels_3, labels_4, labels_5, labels_7, labels_8))
        test_data = data_6
        test_labels = labels_6
        print(f'i: {i}, initiate data finished.')
    elif i == 7:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_2, data_3, data_4, data_5, data_6, data_8))
        train_labels = np.hstack((labels_0, labels_1, labels_2, labels_3, labels_4, labels_5, labels_6, labels_8))
        test_data = data_7
        test_labels = labels_7
        print(f'i: {i}, initiate data finished.')
    elif i == 8:
        print(f'i: {i}, initiate data.')
        train_data = np.vstack((data_0, data_1, data_2, data_3, data_4, data_5, data_6, data_7))
        train_labels = np.hstack((labels_0, labels_1, labels_2, labels_3, labels_4, labels_5, labels_6, labels_7))
        test_data = data_8
        test_labels = labels_8
        print(f'i: {i}, initiate data finished.')
    
    print('train_data.shape: {}'.format(train_data.shape))
    print('train_labels.shape: {}'.format(train_labels.shape))
    print('test_data.shape: {}'.format(test_data.shape))
    print('test_labels.shape: {}'.format(test_labels.shape))

    print(f'generate UserIdentify{i}_TRAIN.ts begin')
    append_train_string = initTimeNetData(train_data, train_labels, 10)
    append_to_file('template.ts', f'UserIdentify{i}_TRAIN.ts', append_train_string)
    print(f'generate UserIdentify{i}_TRAIN.ts finished.')
    print(f'generate UserIdentify{i}_TEST.ts begin')
    append_test_string = initTimeNetData(test_data, test_labels, 10)
    append_to_file('template.ts', f'UserIdentify{i}_TEST.ts', append_test_string)
    print(f'generate UserIdentify{i}_TEST finished.ts')

i: 0, initiate data.
i: 0, initiate data finished.
train_data.shape: (1177115, 7)
train_labels.shape: (1177115,)
test_data.shape: (78978, 7)
test_labels.shape: (78978,)
generate UserIdentify0_TRAIN.ts begin
generate UserIdentify0_TRAIN.ts finished.
generate UserIdentify0_TEST.ts begin
generate UserIdentify0_TEST finished.ts
i: 1, initiate data.
i: 1, initiate data finished.
train_data.shape: (1158543, 7)
train_labels.shape: (1158543,)
test_data.shape: (97550, 7)
test_labels.shape: (97550,)
generate UserIdentify1_TRAIN.ts begin
generate UserIdentify1_TRAIN.ts finished.
generate UserIdentify1_TEST.ts begin
generate UserIdentify1_TEST finished.ts
i: 2, initiate data.
i: 2, initiate data finished.
train_data.shape: (1115190, 7)
train_labels.shape: (1115190,)
test_data.shape: (140903, 7)
test_labels.shape: (140903,)
generate UserIdentify2_TRAIN.ts begin
generate UserIdentify2_TRAIN.ts finished.
generate UserIdentify2_TEST.ts begin
generate UserIdentify2_TEST finished.ts
i: 3, initiate data.